In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
import random
import re
from scipy.sparse import csr_matrix
from datasketch import MinHash,MinHashLSH


In [2]:
df =pd.read_csv("tweets_2022_abril_junio.csv")
df

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0
...,...,...,...,...,...,...
4594975,1526652300709679104,2022-05-17 19:54:06 UTC,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4594976,1526641118460334080,2022-05-17 19:09:40 UTC,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4594977,1526738292011462657,2022-05-18 01:35:48 UTC,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0
4594978,1526855280151056386,2022-05-18 09:20:41 UTC,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0


In [35]:
subconjunto = df.copy()
s = 0.4
k = 4

## Limpieza

In [36]:
spanish_stopwords = stopwords.words('spanish')

In [37]:
frase = "RT @RudyPetrikowski: ⚠️ Ayúdenme por favor a conseguir la información de las normas o artículos que la CC ha rechazado.\n\nEsto es muy import…"
def eliminar_stopwords(frase):
    palabras = frase.split(" ")
    retorno = []
    for palabra in palabras:
        if not palabra in spanish_stopwords:
            retorno.append(palabra)
    return " ".join(retorno)
def procesar_frase(frase):
    if (frase.startswith("RT")):
        frase = ' '.join(frase.split(": ")[1:])
    
    # sin hashatgs
    frase = re.sub(r"#\w+", "",frase)
    # sin url
    frase = re.sub(r"https\S+", "",frase)
    # sin emojis
    patron_emoji = re.compile("[\U0001F600-\U0001F64F]|[\U0001F300-\U0001F5FF]|[\U0001F680-\U0001F6FF]|[\U00002600-\U000027BF]|[\U0001F900-\U0001F9FF]|[\U0001F1E0-\U0001F1FF]|[\U0001F1F2-\U0001F1F4]|[\U0001F1E6-\U0001F1FF]")
    frase = re.sub(patron_emoji, '', frase)
    # sin espacios
    frase = re.sub(r"\s+", " ",frase)
    # eliminar tildes
    frase = frase.replace("á","a")
    frase = frase.replace("é","e")
    frase = frase.replace("í","i")
    frase = frase.replace("ó","o")
    frase = frase.replace("ú","u")

    frase = eliminar_stopwords(frase)
    
    # minusculas
    frase = frase.lower()
    frase = frase.strip()
    return frase

procesar_frase(frase)

'️ ayudenme favor conseguir informacion normas articulos cc rechazado. esto import…'

In [38]:
subconjunto = subconjunto.drop_duplicates(["screen_name","text"]).reset_index(drop=True)
subconjunto

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...,0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...,0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...,0,0
3,1512186335754301446,2022-04-07 21:51:32 UTC,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...,0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...,0,0
...,...,...,...,...,...,...
4549967,1526652300709679104,2022-05-17 19:54:06 UTC,Alebarrera74,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4549968,1526641118460334080,2022-05-17 19:09:40 UTC,gigita29bq,RT @DanielAbelLope1: @tere_marinovic 😡🤮😡🤮 VIEJ...,0,0
4549969,1526738292011462657,2022-05-18 01:35:48 UTC,Elizabe81480339,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0
4549970,1526855280151056386,2022-05-18 09:20:41 UTC,CastilloNafla,RT @Gonz1Gorjeperez: @tere_marinovic https://t...,0,0


In [39]:
agrupado = subconjunto.groupby('screen_name').agg({'text': list})
agrupado['indices'] = subconjunto.groupby('screen_name').apply(lambda x: x.index.tolist()).values

In [40]:
agrupado["indices"]

screen_name
0000000000000JL    [487478, 822212, 911988, 2893127, 2927508, 344...
00001009082AA                                              [3330451]
0001Andino                                                 [4298319]
000Pancho000                                               [3209962]
00132MC                                                    [3374997]
                                         ...                        
zzeendo            [701805, 758247, 2201849, 2751376, 3423484, 34...
zzmatia1           [440698, 654322, 922384, 1160316, 1455774, 176...
zzonck             [2477, 150688, 293644, 302268, 356696, 410598,...
zztoprock1         [94113, 97385, 337760, 457223, 540458, 740315,...
zzurita                         [1284584, 1979972, 2029902, 3700944]
Name: indices, Length: 208139, dtype: object

### Intento 1
Calcular similitudes directo (Lento)

In [8]:
# def Jaccard(set1, set2):
#     interseccion = len(set1.intersection(set2))
#     union = len(set1.union(set2))
#     if (union == 0):
#         return 0
#     return (1 - (interseccion / union))


In [9]:
# similarity_scores = []
# for i in range(subconjunto.shape[0]):
#     for j in range(i+1, subconjunto.shape[0]):
#         user1 = subconjunto.at[i, 'screen_name']
#         user2 = subconjunto.at[j, 'screen_name']
#         if user1 != user2:
#             shingles1 = subconjunto.at[i, 'text']
#             shingles2 = subconjunto.at[j, 'text']
#             similarity = Jaccard(set(shingles1), set(shingles2))
#             if similarity <= s:
#                 similarity_scores.append((user1, user2, similarity))

In [10]:
# for i in similarity_scores:
#     if i[2] <= s:
#         print(i)

In [11]:
# subconjunto[subconjunto["screen_name"]=="h0l4d4ni3l4"]

In [12]:
# subconjunto[subconjunto["screen_name"]=="Valegonsep"].iloc[0].name

### Intento 2 (Mejorable)
Matriz de similitud

In [13]:
# def crear_hash(a, b, primo, modulo):
#     def f(x):
#         return ((a * x + b) % primo) % modulo
#     return f

In [14]:
# # Contamos el total de elementos diferentes
# un = set()
# for i in range(subconjunto.shape[0]):
#         shingl = set(subconjunto.at[i, 'text'])
#         un = un.union(shingl)

# n_elementos = len(un)
# n_elementos

In [15]:
# # Encontramos un primo mayor que n 
# def es_primo(num):
#     if num < 2:
#         return False
#     for i in range(2, int(num**0.5) + 1):
#         if num % i == 0:
#             return False
#     return True


# def siguiente_primo(n):
#     siguiente = n + 1
#     while True:
#         if es_primo(siguiente):
#             return siguiente
#         siguiente += 1


# numero = 15512
# prox_primo = siguiente_primo(numero)
# prox_primo

In [16]:
# h = []
# num_hash = 50
# for i in range(num_hash):
#     a = random.randint(1,prox_primo-1)
#     b = random.randint(1,prox_primo-1)
#     h.append(crear_hash(a,b,prox_primo,n_elementos))

In [17]:
# n_tweets = subconjunto.shape[0]
# FH = [[n_elementos+1 for j in range(n_tweets)] for i in range(num_hash)]

# for i in range(subconjunto.shape[0]):
#         shingl = set(subconjunto.at[i, 'text'])
#         un = un.union(shingl)
#         sorted_union = sorted(un)

In [18]:
# sorted_union[0] not in subconjunto.at[0, 'text']

In [19]:
# sorted_union[0] 

In [20]:
# for e in range(0,len(sorted_union)): 
#     ### iteramos cada columna, correspondiente a la cantidad de textos (tweets en nuestro caso)  
#     for i in range(0,n_tweets):
#         ### si el elemento e no está en el texto i, no hacemos nada
#         if sorted_union[e] not in subconjunto.at[i, 'text']:
#             pass
#         ### si el elemento si está, computamos los valores de h(e) y 
#         else:
#             for j in range(0,num_hash):
#                 FH[j][i] = min(FH[j][i],h[j](e))

In [21]:
# for row in FH:
#     print(row)  

In [22]:
# # Tweets 1 y 127
# cuenta = 0
# print(' 0     126 ')
# print('---------')  
# for row in FH:
#     if row[0]==row[127]:
#         cuenta += 1
#     print('|'+str(row[0])+'|   |'+str(row[127])+'|')  
# print('proporcion de la fila: '+str(cuenta/num_hash))

In [23]:
# def similitud_de_pares(id, ogs, l):
#     persona = subconjunto.at[id,"screen_name"] 
#     datata_person = subconjunto[subconjunto["screen_name"]== persona]
#     n_tweet_persona = datata_person.shape[0]

#     similares = {}
#     for e in range(n_tweets):
#         persona2 =subconjunto.at[e,"screen_name"]
#         if persona != persona2:
#             similares[persona2] = (0,0)

#     for i in range(n_tweet_persona):
#         n_twet = datata_person.iloc[i].name
#         for e in range(n_tweets):
#             persona2 =subconjunto.at[e,"screen_name"]
#             if persona != persona2:
#                 cuenta_persona = similares[persona2][1]
#                 colisiones_persona = similares[persona2][1]
#                 cuenta = 0
#                 for row in FH:
#                     if row[n_twet]==row[e]:
#                         cuenta += 1
                
#                 cuenta_persona +=1
#                 if cuenta/num_hash >= s:
#                     colisiones_persona += 1
#                 similares[persona2] = (cuenta_persona,colisiones_persona)
    

#     for i in similares.keys():
#         if similares[i][1] != (0,0):
#             if similares[i][1] >= l:
#                 if similares[i][0]>= k:
#                     ogs[persona].append(i)
#     return ogs

In [24]:
# personas = subconjunto['screen_name'].unique().tolist()
# ogs = {}
# for persona in personas:
#     ogs[persona]=[]

In [25]:
# personas_unicas = subconjunto.drop_duplicates(subset='screen_name', keep='first')

# for i in range(len(personas_unicas)):
#     indice = personas_unicas.iloc[i].name
#     ogs = similitud_de_pares(indice, ogs, 3)

In [26]:
# ogs

## Intento 3 (Definitivo)
Usamos librerias junto con el concepto anterior

In [41]:
def shingle(frase , tamaño_ventana):
    shingles = []
    for i in range(len(frase) - tamaño_ventana +1):
        shingle = ' '.join(frase[i:i+tamaño_ventana])
        shingles.append(shingle)
    return shingles

subconjunto["text"] = subconjunto["text"].apply(procesar_frase)
subconjunto["text"] = subconjunto["text"].str.split().apply(lambda x: np.array(x)).apply(lambda x: shingle(x,k))

In [46]:
subconjunto = subconjunto[subconjunto['text'].apply(lambda x: len(x) > 0)]

In [47]:
subconjunto['text']

0          [tras casi 50 años, casi 50 años golpe,, 50 añ...
1          [mañana jueves 18hrs. comienza, jueves 18hrs. ...
2          [aqui aporte @tere_marinovic respecto, aporte ...
4          [ante circulacion noticias falsas,, circulacio...
5          [@rodrigo_logan pesima respuesta. deberias, pe...
                                 ...                        
4549959    [@ignacioachurra nuestra bandera fragmentada.,...
4549960    [@tere_marinovic el nuevo ministerio, el nuevo...
4549961    [@tere_marinovic el nuevo ministerio, el nuevo...
4549962    [@tere_marinovic el nuevo ministerio, el nuevo...
4549963    [@tere_marinovic el nuevo ministerio, el nuevo...
Name: text, Length: 4019415, dtype: object

In [48]:
subconjunto

,id,created_at,screen_name,text,favorite_count,retweet_count
0,1512186166438637582,2022-04-07 21:50:51 UTC,h0l4d4ni3l4,"[tras casi 50 años, casi 50 años golpe,, 50 añ...",0,0
1,1512186202367045642,2022-04-07 21:51:00 UTC,Claudio70932894,"[mañana jueves 18hrs. comienza, jueves 18hrs. ...",0,0
2,1512186287284924418,2022-04-07 21:51:20 UTC,Cesar_A_RR,"[aqui aporte @tere_marinovic respecto, aporte ...",0,0
4,1512186407841767424,2022-04-07 21:51:49 UTC,GQuelluen,"[ante circulacion noticias falsas,, circulacio...",0,0
5,1512186484731744256,2022-04-07 21:52:07 UTC,iramirezpavez,"[@rodrigo_logan pesima respuesta. deberias, pe...",0,0
...,...,...,...,...,...,...
4549959,1526607670974685186,2022-05-17 16:56:46 UTC,clabarra01,"[@ignacioachurra nuestra bandera fragmentada.,...",0,0
4549960,1526560610422558722,2022-05-17 13:49:46 UTC,Anitace,"[@tere_marinovic el nuevo ministerio, el nuevo...",0,0
4549961,1526563563749314560,2022-05-17 14:01:30 UTC,EllaResponde,"[@tere_marinovic el nuevo ministerio, el nuevo...",0,0
4549962,1526678643421786112,2022-05-17 21:38:47 UTC,CeaKrisil,"[@tere_marinovic el nuevo ministerio, el nuevo...",0,0


In [49]:
def m_caracteristica(subconjun, sample_size):
    tweets = subconjun
    tweets = tweets.sample(sample_size)
    ides = list(tweets["id"])
    shingles = tweets["text"].to_numpy()
    mlb = MultiLabelBinarizer(sparse_output=True)
    return (mlb.fit_transform(shingles),ides)

In [50]:
pares = []
for i in range(14):
    matrix, ides = m_caracteristica(subconjunto, 300000)
    pares.append((matrix, ides))

In [53]:
n_pm= 119

def minhash_vec(sparce_vector):
    minhash = MinHash(n_pm)
    for i in sparce_vector.indices:
        minhash.update(str(i).encode("utf8"))
    return minhash

In [ ]:
primer_par = pares[0]
matrix = primer_par[0]
ides = primer_par[1]

In [54]:
thrld = 0.4
lsh = MinHashLSH(thrld,n_pm)
list_hash = []
for i in range(matrix.shape[0]):
    list_hash.append((i,minhash_vec(matrix[i])))

with lsh.insertion_session() as session:
    for i , minhash in list_hash:
        session.insert(i, minhash)

In [23]:
len(list_hash)

300000

In [26]:
id_tweet = ides[1000]
print(id_tweet)
m = minhash_vec(matrix[1000])
aprox = lsh.query(m)
aprox

1511770098343755780


[147459,
 4103,
 74257,
 230426,
 8731,
 168989,
 168489,
 98876,
 144455,
 79944,
 280137,
 36948,
 221277,
 108639,
 103008,
 185448,
 146030,
 76916,
 38520,
 35450,
 81557,
 175766,
 289432,
 174751,
 231077,
 127657,
 13485,
 233134,
 279226,
 104127,
 281287,
 24781,
 152784,
 193233,
 297173,
 214744,
 48863,
 31978,
 241900,
 50927,
 146676,
 120054,
 6393,
 205056,
 120578,
 63750,
 269579,
 50457,
 129310,
 97567,
 252195,
 246053,
 80680,
 211758,
 165692,
 38731,
 167755,
 73037,
 251213,
 270669,
 214361,
 13168,
 55666,
 241523,
 128372,
 178042,
 197513,
 175501,
 197008,
 63900,
 252329,
 74154,
 42924,
 213421,
 262572,
 289215,
 231363,
 103368,
 96719,
 198607,
 260560,
 1000,
 19945,
 163817,
 185338,
 216061]

In [27]:
idd = 1000
m = minhash_vec(matrix[idd])
aprox = lsh.query(m)

subconjunto.iloc[idd]["text"]

reps = {}
for i in aprox:
    twet = subconjunto.iloc[i]
    if not twet["screen_name"] in reps.keys():
        reps[twet["screen_name"]] =1
    else:
        reps[twet["screen_name"]] +=1


In [32]:
defi = []
for i in reps.keys():
    if reps[i] >= 2:
        defi.append(i)

In [33]:
defi

['sdalmazzo']

In [146]:
m = minhash_vec(matrix[19])
lsh.query(m)

[0, 2, 6, 7, 8, 9, 13, 14, 16, 17, 18, 19]